In [1]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from urllib.error import URLError, HTTPError
import pandas as pd
from time import sleep

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.6613.119 Safari/537.36'}

#Necessário para ter as siglas dos times da NBA
times = {'Los Angeles Lakers' : 'LAL',
        'Milwaukee Bucks' : 'MIL',
        'Boston Celtics' : 'BOS',
        'Philadelphia 76ers' : 'PHI',
        'Portland Trail Blazers' : 'POR',
        'Detroit Pistons' : 'DET',
        'Denver Nuggets' : 'DEN',
        'Dallas Mavericks' : 'DAL',
        'Houston Rockets' : 'HOU',
        'San Antonio Spurs' : 'SAS',
        'New Jersey Nets' : 'NJN',
        'Utah Jazz' : 'UTA',
        'Golden State Warriors' : 'GSW',
        'Chicago Bulls' : 'CHI',
        'Cleveland Cavaliers' : 'CLE',
        'Indiana Pacers' : 'IND',
        'Phoenix Suns': 'PHO',
        'Los Angeles Clippers' : 'LAC',
        'Atlanta Hawks' : 'ATL',
        'Washington Bullets' : 'WSB',
        'New York Knicks' : 'NYK',
        'Seattle SuperSonics' : 'SEA',
        'Toronto Raptors' : 'TOR',
        'Miami Heat' : 'MIA',
        'Brooklyn Nets' : 'BRK',
        'Charlotte Hornets' : 'CHO',
        'Washington Wizards' : 'WAS',
        'Orlando Magic' : 'ORL',
        'Oklahoma City Thunder' : 'OKC',
        'Memphis Grizzlies' : 'MEM',
        'Sacramento Kings' : 'SAC',
        'New Orleans Pelicans' : 'NOP',
        'Minnesota Timberwolves' : 'MIN',
        'San Diego Clippers' : 'SDC',
        'Kansas City Kings' : 'KCK',
        'Charlotte Bobcats' : 'CHA',
        'Vancouver Grizzlies' : 'VAN',
        'New Orleans Hornets' : 'NOH',
        'New Orleans/Oklahoma City Hornets' : 'NOK'}

#Intervalos de anos dos dados das temporadas que serão extraidos
anos = list(range(1980,2025))
anos = [2025]

# Scrapping de todas as temporadas

In [9]:
def scrapping_times(url):
    try:
        req = Request(url, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)

    table_e = soup.find('table', attrs={'id':'confs_standings_E'})
    if table_e == None:
        table_e = soup.find('table', attrs={'id':'divs_standings_E'})
    
    
    th_list = table_e.find('tbody').find_all('th')
    dados_time_e = [x.find('a').get_text().strip() for x in th_list if x.find('a')!=None]

    table_w = soup.find('table', attrs={'id':'confs_standings_W'})
    if table_w == None:
        table_w = soup.find('table', attrs={'id':'divs_standings_W'})
        
    th_list = table_w.find('tbody').find_all('th')

        
    th_list = table_w.find('tbody').find_all('th')

    dados_time_w = [x.find('a').get_text().strip() for x in th_list if x.find('a')!=None]
    
    return dados_time_e + dados_time_w

In [10]:
def scrapping_dados_jogos(url, time_casa, season):
    try:
        req = Request(url, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)

    except Exception as e:
        print(e)

    dados_jogos = []

    table_games = soup.find('table', attrs={'id':'games'})
    tr_keys = table_games.find('thead').find_all('th')
    keys = [x.get_text().strip() for x in tr_keys]

    tbody_games = table_games.find('tbody')
    tr_list = tbody_games.find_all('tr')

    for tr_inst in tr_list:
        if(tr_inst.get('class') == None):
            dados_jogo = {}
            dados_jogo['g'] = tr_inst.find('th').get_text().strip()
            dados_jogo['season'] = season
            dados_jogo['date_game'] =  tr_inst.find('td', attrs={'data-stat':'date_game'}).find('a').get_text().strip()
            if  tr_inst.find('td', attrs={'data-stat':'game_start_time'}) != None :
                dados_jogo['game_start_time'] = tr_inst.find('td', attrs={'data-stat':'game_start_time'}).get_text().strip()
            else:
                dados_jogo['game_start_time'] = None
            dados_jogo['link'] = 'https://www.basketball-reference.com' + tr_inst.find('td', attrs={'data-stat':'box_score_text'}).find('a').get('href')
            dados_jogo['team_1'] = time_casa
            team_2 = tr_inst.find('td', attrs={'data-stat':'opp_name'}).find('a').get_text().strip()
            dados_jogo['team_2'] = times[team_2]
            dados_jogo['home'] = 0 if tr_inst.find('td', attrs={'data-stat':'game_location'}).get_text().strip() != '@' else 1
            dados_jogo['game_result'] = tr_inst.find('td', attrs={'data-stat':'game_result'}).get_text().strip()
            dados_jogo['pts'] = tr_inst.find('td', attrs={'data-stat':'pts'}).get_text().strip()
            dados_jogo['opp_pts'] = tr_inst.find('td', attrs={'data-stat':'opp_pts'}).get_text().strip()
            dados_jogo['wins'] = tr_inst.find('td', attrs={'data-stat':'wins'}).get_text().strip()
            dados_jogo['losses'] = tr_inst.find('td', attrs={'data-stat':'losses'}).get_text().strip()
            dados_jogo['game_streak'] = tr_inst.find('td', attrs={'data-stat':'game_streak'}).get_text().strip()
            dados_jogo['game_remarks'] = tr_inst.find('td', attrs={'data-stat':'game_remarks'}).get_text().strip()

            dados_jogos.append(dados_jogo)
            
            
    return dados_jogos

In [20]:
dados = []
for index_ano, ano in enumerate(anos):
    # Capturando os times daquela temporada (Necessário, pois franquias podem ser vendidas)
    url_dados_time = f'https://www.basketball-reference.com/leagues/NBA_{ano}_standings.html'
    
    try:
        times_participantes = scrapping_times(url_dados_time)
    except:
        print("Erro na captura de times da temporada {}".format(ano))
    
    print('Capturados os times da temporada {}'.format(ano))
    
    siglas_times_participantes = [times[x] for x in times_participantes]
    
    for index_sigla, sigla in enumerate(siglas_times_participantes):
        url_dados_jogos = f'https://www.basketball-reference.com/teams/{sigla}/{ano}_games.html'
        
        try:
            dados_time = scrapping_dados_jogos(url_dados_jogos, sigla, ano)
        except:
            print("Erro no time {} na temporada {}".format(sigla, ano))
        
        print('Capturado os jogos do time {} (index {}) da temporada {} (index {})'.format(sigla, index_sigla, ano, index_ano))
        sleep(2.5)
        dados.append(dados_time)

    print('Capturados os jogos da temporada {} (index {})'.format(ano, index_ano))

Capturados os times da temporada 2025
Capturado os jogos do time CLE (index 0) da temporada 2025 (index 0)
Capturado os jogos do time BOS (index 1) da temporada 2025 (index 0)
Capturado os jogos do time NYK (index 2) da temporada 2025 (index 0)
Capturado os jogos do time IND (index 3) da temporada 2025 (index 0)
Capturado os jogos do time MIL (index 4) da temporada 2025 (index 0)
Capturado os jogos do time DET (index 5) da temporada 2025 (index 0)
Capturado os jogos do time ORL (index 6) da temporada 2025 (index 0)
Capturado os jogos do time ATL (index 7) da temporada 2025 (index 0)
Capturado os jogos do time CHI (index 8) da temporada 2025 (index 0)
Capturado os jogos do time MIA (index 9) da temporada 2025 (index 0)


KeyboardInterrupt: 

In [12]:
dados_jogos = []
for dado in dados:
    for jogo_dado in dado:
        dados_jogos.append(jogo_dado)
dataframe = pd.DataFrame(dados_jogos)

dataframe.head()

,g,season,date_game,game_start_time,link,team_1,team_2,home,game_result,pts,opp_pts,wins,losses,game_streak,game_remarks
0,1,2025,"Wed, Oct 23, 2024",7:30p,https://www.basketball-reference.com/boxscores...,CLE,TOR,1,W,136,106,1,0,W 1,
1,2,2025,"Fri, Oct 25, 2024",7:30p,https://www.basketball-reference.com/boxscores...,CLE,DET,0,W,113,101,2,0,W 2,
2,3,2025,"Sat, Oct 26, 2024",7:00p,https://www.basketball-reference.com/boxscores...,CLE,WAS,1,W,135,116,3,0,W 3,
3,4,2025,"Mon, Oct 28, 2024",7:30p,https://www.basketball-reference.com/boxscores...,CLE,NYK,1,W,110,104,4,0,W 4,
4,5,2025,"Wed, Oct 30, 2024",7:00p,https://www.basketball-reference.com/boxscores...,CLE,LAL,0,W,134,110,5,0,W 5,


In [ ]:
# Retirando os valores com home=1 para que o team_2 seja sempre o time visitante
dataframe = dataframe.drop(dataframe[dataframe['home'] == 0].index)
dataframe.to_csv('./dados_jogos_pt2_2025.csv', index=False)

In [14]:
def scrapping_jogo(url, dados_partida):
    try:
        req = Request(url, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)

    except Exception as e:
        print(e)

    # Capturando os dados da partida do time 1
    table_team1 = soup.find('table', attrs={'id':'box-{}-game-basic'.format(dados_partida['team_1'])})
    if table_team1 == None and dados_partida['team_1'] == 'CHO':
        dados_partida['team_1'] = 'CHH'
        table_team1 = soup.find('table', attrs={'id':'box-{}-game-basic'.format(dados_partida['team_1'])})
    
    th_list_team1 = table_team1.find('thead').find_all('tr')[1].find_all('th')

    keys_basic =  [x.get_text().strip().lower() for x in th_list_team1]

    td_basic_team1 = table_team1.find('tfoot').find('tr').find_all('td')

    i = 1
    for td_inst in td_basic_team1[:-2]:
        key = 'team_1_' + td_inst.get('data-stat').strip()
        dados_partida[key] = td_inst.get_text().strip()
    

    table_advanced_team1 = soup.find('table', attrs={'id':'box-{}-game-advanced'.format(dados_partida['team_1'])})
    
    th_list_team1 = table_advanced_team1.find('thead').find_all('tr')[1].find_all('th')

    keys_advanced =  [x.get_text().strip().lower() for x in th_list_team1]

    td_advanced_team1 = table_advanced_team1.find('tfoot').find('tr').find_all('td')

    i = 1
    for td_inst in td_advanced_team1[:-1]:
        key = 'team_1_' + td_inst.get('data-stat').strip()
        dados_partida[key] = td_inst.get_text().strip()

    # Capturando os dados do time 2
    print("Time 2:", dados_partida['team_2'])
    table_team2 = soup.find('table', attrs={'id':'box-{}-game-basic'.format(dados_partida['team_2'])})
    if table_team2 == None and dados_partida['team_2'] == 'CHO':
        dados_partida['team_2'] = 'CHH'
        table_team2 = soup.find('table', attrs={'id':'box-{}-game-basic'.format(dados_partida['team_2'])})
    #th_list_team2 = table_team2.find('thead').find_all('tr')[1].find_all('th')
    td_basic_team2 = table_team2.find('tfoot').find('tr').find_all('td')

    for td_inst in td_basic_team2[:-2]:
        key = 'team_2_' + td_inst.get('data-stat').strip()
        dados_partida[key] = td_inst.get_text().strip()

    table_advanced_team2 = soup.find('table', attrs={'id':'box-{}-game-advanced'.format(dados_partida['team_2'])})
    #th_list_team2 = table_advanced_team2.find('thead').find_all('tr')[1].find_all('th')

    td_advanced_team2 = table_advanced_team2.find('tfoot').find('tr').find_all('td')
    #print("Teste:", td_advanced_team2)
    for td_inst in td_advanced_team2[:-1]:
        key = 'team_2_' + td_inst.get('data-stat').strip()
        dados_partida[key] = td_inst.get_text().strip()
    

In [15]:
dataframe = pd.read_csv('./dados_jogos_pt2_2025.csv')
dataframe['season'] = dataframe['season'].astype(int)
dataframe = dataframe[dataframe['season']>2000]
dataframe.head()

,g,season,date_game,game_start_time,link,team_1,team_2,home,game_result,pts,opp_pts,wins,losses,game_streak,game_remarks
0,2,2025,"Fri, Oct 25, 2024",7:30p,https://www.basketball-reference.com/boxscores...,CLE,DET,0,W,113,101,2,0,W 2,NaN
1,5,2025,"Wed, Oct 30, 2024",7:00p,https://www.basketball-reference.com/boxscores...,CLE,LAL,0,W,134,110,5,0,W 5,NaN
2,6,2025,"Fri, Nov 1, 2024",7:00p,https://www.basketball-reference.com/boxscores...,CLE,ORL,0,W,120,109,6,0,W 6,NaN
3,8,2025,"Mon, Nov 4, 2024",7:00p,https://www.basketball-reference.com/boxscores...,CLE,MIL,0,W,116,114,8,0,W 8,NaN
4,10,2025,"Fri, Nov 8, 2024",7:30p,https://www.basketball-reference.com/boxscores...,CLE,GSW,0,W,136,117,10,0,W 10,NaN


In [16]:
colunas = list(dataframe.columns)
dados = []
for index, row in dataframe.iterrows():
    dados_jogo = {}
    for coluna in colunas:
        dados_jogo[coluna] = row[coluna]
    dados.append(dados_jogo)

In [17]:
dados[:10]

[{'g': 2,
  'season': 2025,
  'date_game': 'Fri, Oct 25, 2024',
  'game_start_time': '7:30p',
  'link': 'https://www.basketball-reference.com/boxscores/202410250CLE.html',
  'team_1': 'CLE',
  'team_2': 'DET',
  'home': 0,
  'game_result': 'W',
  'pts': 113,
  'opp_pts': 101,
  'wins': 2,
  'losses': 0,
  'game_streak': 'W 2',
  'game_remarks': nan},
 {'g': 5,
  'season': 2025,
  'date_game': 'Wed, Oct 30, 2024',
  'game_start_time': '7:00p',
  'link': 'https://www.basketball-reference.com/boxscores/202410300CLE.html',
  'team_1': 'CLE',
  'team_2': 'LAL',
  'home': 0,
  'game_result': 'W',
  'pts': 134,
  'opp_pts': 110,
  'wins': 5,
  'losses': 0,
  'game_streak': 'W 5',
  'game_remarks': nan},
 {'g': 6,
  'season': 2025,
  'date_game': 'Fri, Nov 1, 2024',
  'game_start_time': '7:00p',
  'link': 'https://www.basketball-reference.com/boxscores/202411010CLE.html',
  'team_1': 'CLE',
  'team_2': 'ORL',
  'home': 0,
  'game_result': 'W',
  'pts': 120,
  'opp_pts': 109,
  'wins': 6,
  'lo

In [18]:
for index_dados_jogo in range(len(dados)):
    dados_jogo_time = dados[index_dados_jogo]
    url_jogo = dados_jogo_time['link']
    print(url_jogo)
    scrapping_jogo(url_jogo, dados_jogo_time)
    print('Capturado dados do jogo (index {})'. format(index_dados_jogo))
    sleep(2.5)

https://www.basketball-reference.com/boxscores/202410250CLE.html
Time 2: DET
Capturado dados do jogo (index 0)
https://www.basketball-reference.com/boxscores/202410300CLE.html
Time 2: LAL
Capturado dados do jogo (index 1)
https://www.basketball-reference.com/boxscores/202411010CLE.html
Time 2: ORL
Capturado dados do jogo (index 2)
https://www.basketball-reference.com/boxscores/202411040CLE.html
Time 2: MIL
Capturado dados do jogo (index 3)
https://www.basketball-reference.com/boxscores/202411080CLE.html
Time 2: GSW
Capturado dados do jogo (index 4)
https://www.basketball-reference.com/boxscores/202411090CLE.html
Time 2: BRK
Capturado dados do jogo (index 5)
https://www.basketball-reference.com/boxscores/202411150CLE.html
Time 2: CHI
Capturado dados do jogo (index 6)
https://www.basketball-reference.com/boxscores/202411170CLE.html
Time 2: CHO
Capturado dados do jogo (index 7)
https://www.basketball-reference.com/boxscores/202411200CLE.html
Time 2: NOP
Capturado dados do jogo (index 8)
h

In [19]:
import pickle

with open('dados_2025.pkl', 'wb') as f:
    pickle.dump(dados, f)    

TypeError: object of type 'NoneType' has no len()